In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [5]:
import tools
tools.FOLDER_ROOT = '/home/renj/Data/beijing 2008'

In [6]:
data = pd.DataFrame()
for fd in ['01', '02', '03','04','05','06','07']:
    d = tools.read_data('2nd_folder', folder = fd, n_rows = None)
    data = pd.concat([data, d])

In [7]:
print len(data), len(data.id.unique())

10206809 2891


In [8]:
sample_data = data
print len(sample_data), len(sample_data.id.unique())

10206809 2891


In [9]:
sample_data.to_pickle('./data/sample_data.pickle')

In [10]:
sample_data = pd.read_pickle('./data/sample_data.pickle')

In [11]:
def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    http://boulter.com/gps/distance/
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = 6367000.0 * c
    return m

In [12]:
def over_speed(d, max_speed):
    delt_dist = map(lambda x: haversine(x[0], x[1], x[2], x[3]), zip(d.x[1:], d.y[1:], d.x[:-1], d.y[:-1]))
    dt = pd.to_datetime(d.time)
    s = map(lambda x: x.total_seconds(), np.subtract(dt[1:], dt[:-1]))
    df = pd.DataFrame(zip(range(len(s)), s, delt_dist))
    df['speed'] = np.divide(df[2], df[1])
    df = df.fillna(0)
    #df = df.drop(df[df.speed > max_speed].index)
    #return df
    idx = df[df.speed > max_speed].index+1
    i = 0
    drop_idx = []
    while i < len(idx):
        if i+1 >= len(idx) or idx[i+1]-idx[i] > 5:
            drop_idx.append(idx[i])
        else:
            drop_idx += range(idx[i],idx[i+1])
            i += 1
        i += 1
    return drop_idx

In [13]:
def speed(d):
    delt_dist = map(lambda x: haversine(x[0], x[1], x[2], x[3]), zip(d.x[1:], d.y[1:], d.x[:-1], d.y[:-1]))
    dt = pd.to_datetime(d.time)
    s = map(lambda x: x.total_seconds(), np.subtract(dt[1:], dt[:-1]))
    df = pd.DataFrame(zip(range(len(s)), s, delt_dist))
    df['speed'] = np.divide(df[2], df[1])
    df = df.fillna(0)
    return df

In [ ]:
i = 0
MAX_SPEED = 60
fined_data = pd.DataFrame()
for id in sample_data.id.unique():
    print i, id
    d = sample_data[sample_data.id == id]
    if len(d) <= 1: continue
    drop_idx = over_speed(d, MAX_SPEED)
    while len(drop_idx) != 0:
        d = d.drop(drop_idx)
        d.index = range(len(d))
        drop_idx = over_speed(d, MAX_SPEED)
    assert len(drop_idx) == 0
    fined_data = pd.concat([fined_data, d])
    i += 1

0 6665
1 7105
2 8717
3 7146
4 6656
5 9109
6 1277
7 6275
8 5099
9 2560
10 2669
11 3557
12 8662
13 366
14 3015
15 4798
16 9754
17 1131
18 2237
19 5075
20 3579
21 5860
22 3781
23 8179
24 1509
25 4867
26 7173
27 2313
28 7922
29 9539
30 9468
31 1838
32 1741
33 4396
34 7056
35 8474
36 7382
37 4265
38 5444
39 4819
40 6818
41 8654
42 6810
43 5856
44 5478
45 5854
46 4115
47 8525
48 6790
49 3872
50 6211
51 4578
52 5071
53 6358
54 2910
55 2884
56 4461
57 2172
58 715
59 9944
60 9579
61 9207
62 7427
63 6903
64 5208
65 8652
66 1563
67 5414
68 6607
69 919
70 3953
71 9948
72 650
73 8291
74 9945
75 4990
76 7971
77 950
78 4112
79 9580
80 750
81 2780
82 4854
83 3799
84 8657
85 5894
86 1757
87 2193
88 9946
89 676
90 8001
91 9415
92 6942
93 8463
94 5316
95 8860
96 8801
97 534
98 9952
99 1374
100 9949
101 7630
102 1775
103 6814
104 9950
105 5867
106 3895
107 2154
108 7919
109 4863
110 9578
111 7195
112 4177
113 1827
114 8126
115 5460
116 7653
117 7430
118 8554
119 5987
120 8789
121 7883
122 2486
123 6464
12

In [ ]:
len(fined_data)

In [ ]:
len(fined_data.id.unique())

In [ ]:
len(fined_data)

In [ ]:
fined_data.to_pickle('./data/fined_data.pickle')

In [2]:
fined_data = pd.read_pickle('./data/fined_data.pickle')

In [3]:
d = fined_data.copy()

In [4]:
d.index = range(1, len(d)+1)
d.time = pd.to_datetime(d.time)
delta_time = np.subtract(d.time[1:], d.time[:-1])
delta_id = np.subtract(d.id[1:], d.id[:-1])
split_id = delta_id[delta_id != 0].index
split_time = delta_time[delta_time > np.timedelta64(10,'m')].index
split_all = pd.Series([1] + list(split_id) + list(split_time) + [len(d)]).unique()
split_all.sort()
window = np.subtract(split_all[1:], split_all[:-1])
window[-1] += 1
l_id = []
idx = 1
for i in window:
    l_id += [idx] * i
    idx += 1
assert len(d) == sum(window)
d['tid'] = l_id

In [5]:
d.tid.max()

122060

In [7]:
a = d.id.groupby(d.tid).count()

In [8]:
a.sort()

In [9]:
a.describe()

count    122060.000000
mean         82.547772
std         592.787708
min           1.000000
25%           6.000000
50%          24.000000
75%          71.000000
max       50008.000000
Name: id, dtype: float64

In [10]:
a[-50000:].describe()

count    50000.000000
mean       185.010220
std        916.471848
min         35.000000
25%         54.000000
50%         89.000000
75%        163.000000
max      50008.000000
Name: id, dtype: float64

In [11]:
sample_tids = list(a.index)

In [12]:
d.index = d.tid
d = d.ix[sample_tids]

In [13]:
print len(d)
print len(fined_data)

10075781
10075781


In [14]:
tids = d.index.unique()


In [15]:
for num in [80000,60000,40000,20000,10000]:
    sample_tids = random.sample(tids, num)
    sample = d.ix[sample_tids].copy()
    shift = dict(zip(sample_tids, range(1, len(sample_tids)+1)))
    new_index = []
    for i in sample.index:
        new_index.append(shift[i])
    sample.index = new_index
    sample.tid = sample.index
    a = sample.id.groupby(sample.tid).count()
    print a.describe()
    sample.to_pickle('./data/Data_short_'+str(num)+'.pickle')

count    80000.000000
mean        85.926350
std        669.389744
min          1.000000
25%          6.000000
50%         24.000000
75%         71.000000
max      50008.000000
Name: id, dtype: float64
count    60000.000000
mean        82.647983
std        599.038514
min          1.000000
25%          6.000000
50%         23.000000
75%         70.000000
max      50008.000000
Name: id, dtype: float64
count    40000.000000
mean        79.594950
std        450.611897
min          1.000000
25%          6.000000
50%         24.000000
75%         70.000000
max      28555.000000
Name: id, dtype: float64
count    20000.000000
mean        83.842350
std        566.428589
min          1.000000
25%          6.000000
50%         24.000000
75%         74.000000
max      34060.000000
Name: id, dtype: float64
count    10000.000000
mean        84.433000
std        648.036484
min          1.000000
25%          6.000000
50%         24.000000
75%         71.000000
max      33139.000000
Name: id, dtype: flo

In [37]:
"""
generate long trajectory data
"""
for num in [100, 500, 1000, 5000, 10000]:
    sample_tids = list(a[(-1*num):].index)
    sample = d.ix[sample_tids].copy()
    shift = dict(zip(sample_tids, range(1, len(sample_tids)+1)))
    new_index = []
    for i in sample.index:
        new_index.append(shift[i])
    sample.index = new_index
    sample.tid = sample.index
    sample.to_pickle('./data/Data_long_'+str(num)+'.pickle')

In [44]:
tids = list(a[-50000:].index)
for num in [100, 500, 1000, 5000, 10000]:
    sample_tids = random.sample(tids, num)
    sample = d.ix[sample_tids].copy()
    shift = dict(zip(sample_tids, range(1, len(sample_tids)+1)))
    new_index = []
    for i in sample.index:
        new_index.append(shift[i])
    sample.index = new_index
    sample.tid = sample.index
    a = sample.id.groupby(sample.tid).count()
    print a.describe()
    sample.to_pickle('./data/Data_middle_'+str(num)+'.pickle')

count     100.000000
mean      148.110000
std       194.596599
min        35.000000
25%        57.750000
50%       106.000000
75%       186.000000
max      1840.000000
Name: id, dtype: float64
count      500.000000
mean       181.894000
std        817.926585
min         35.000000
25%         56.750000
50%         92.000000
75%        157.250000
max      15339.000000
Name: id, dtype: float64
count     1000.000000
mean       228.409000
std       1392.534787
min         35.000000
25%         55.000000
50%         88.500000
75%        151.000000
max      34060.000000
Name: id, dtype: float64
count     5000.000000
mean       185.318800
std        833.467405
min         35.000000
25%         54.000000
50%         90.000000
75%        162.000000
max      32450.000000
Name: id, dtype: float64
count    10000.000000
mean       189.692300
std       1107.428165
min         35.000000
25%         54.000000
50%         89.000000
75%        161.000000
max      49095.000000
Name: id, dtype: float64


In [504]:
d = pd.read_pickle('./data/Data_100.pickle')
len(d)

15601

In [482]:
sparse = dict()

In [484]:
d.head()

,id,time,x,y,tid
1,2911,2008-02-05 11:11:49,116.28820,39.83573,1
1,2911,2008-02-05 11:12:51,116.28819,39.83581,1
1,2911,2008-02-05 11:14:09,116.28819,39.83581,1
1,2911,2008-02-05 11:15:30,116.28819,39.83581,1
1,2911,2008-02-05 11:16:53,116.28647,39.83556,1


In [486]:
print d.x.min(), d.x.max(), d.y.min(), d.y.max()

116.05916 117.82969 39.58215 40.89593


In [491]:
len(range( int(d.x.min()*1000), int(d.x.max()*1000)))

1770

In [490]:
len(range( int(d.y.min()*1000), int(d.y.max()*1000)))

1313

In [492]:
len(d[d.tid == 1])

37

In [511]:
s = 'var lines = ['
for tid in d.tid.unique():
    s += '[\n'
    for p in d[d.tid == tid].values:
        s += 'new BMap.Point(%f,%f),\n' % (p[2], p[3])
    s += '],\n'
s += ']'
print s

In [506]:
with open('./data/data.js','w') as f:
    f.write(s)
    

In [515]:
s = 'var points = ['
for p in d.values:
    s += '{"lng":%f,"lat":%f,"count":1},\n' % (p[2], p[3])
s += '];'

with open('./data/data_points.js','w') as f:
    f.write(s)


In [514]:
s

'var points = [{"lng":116.484330,"lat":39.904880,"count":1},{"lng":116.484330,"lat":39.904880,"count":1},{"lng":116.522790,"lat":39.908570,"count":1},{"lng":116.514950,"lat":39.922070,"count":1},{"lng":116.530840,"lat":39.938730,"count":1},{"lng":116.523870,"lat":39.936390,"count":1},{"lng":116.520330,"lat":39.924670,"count":1},{"lng":116.520290,"lat":39.924650,"count":1},{"lng":116.520260,"lat":39.924660,"count":1},{"lng":116.520240,"lat":39.924650,"count":1},{"lng":116.520070,"lat":39.924660,"count":1},{"lng":116.519870,"lat":39.924660,"count":1},{"lng":116.483600,"lat":39.899420,"count":1},{"lng":116.500690,"lat":39.869350,"count":1},{"lng":116.494300,"lat":39.871480,"count":1},{"lng":116.484000,"lat":39.888250,"count":1},{"lng":116.483730,"lat":39.896380,"count":1},{"lng":116.471680,"lat":39.904710,"count":1},{"lng":116.476370,"lat":39.907290,"count":1},{"lng":116.471650,"lat":39.907270,"count":1},{"lng":116.465320,"lat":39.912290,"count":1},{"lng":116.455580,"lat":39.945800,"count